# ACOPF in QCQP Form

ACOPF based on QCQP formulation
This is the 6 bus example from pp. 372-376 of
"Power Generation, Operation, and Control, 3nd Edition",
by Allen. J. Wood and Bruce F. Wollenberg, John Wiley & Sons, NY, Jan 1996.

Formulation is based on:
X. Bai, H. Wei, K. Fujisawa, and Y. Wang, “Semidefinite programming for optimal power flow problems,” International Journal of Electrical Power & Energy Systems, vol. 30, no. 6, pp. 383–392, Jul. 2008.


In [6]:
sbase = 100 # MVA

# Cost Coefficents
a = [213.1, 200.0, 240.0,0,0,0]
b = [11.669, 10.333, 10.833, 0,0,0]*sbase
c = [0.00533, 0.00889, 0.00741, 0,0,0]*sbase^2

# Generation Limits
Pmin = [50, 37.5, 45,0,0,0]/sbase
Pmax = [200, 150, 180,0,0,0]/sbase

Qmin = [-100,-100,-100,0,0,0]/sbase
Qmax = [150,150,120,0,0,0]/sbase

# Load Information
Pload = [0,0,0,100,100,100]/sbase
Qload = [0,0,0,15,15,15]/sbase

# Voltage limits
Vmin = [0.95,0.95,0.95,0.95,0.95,0.95,0.95]
Vmax = [1.07,1.07,1.07,1.07,1.07,1.07]


B = [-11.74791555 4 0 4.705882353 3.112033195 0;
     4 -23.19549683 3.846153846 8 3 4.454342984;
     0 3.846153846 -16.56727017 0 3.170731707 9.615384615
     4.705882353 8 0 -14.63588235 2 0;
     3.112033195 3 3.170731707 2 -14.1377649 3
     0 4.454342984 9.615384615 0 3 -17.0047276 ]

G = [4.006346107 -2 0 -1.176470588 -0.829875519 0
     -2 9.328250814 -0.769230769 -4 -1 -1.559020045
      0 -0.769230769 4.155722326 0 -1.463414634 -1.923076923
      -1.176470588 -4 0 6.176470588 -1 0
      -0.829875519 -1 -1.463414634 -1 5.293290153 -1
       0 -1.559020045 -1.923076923 0 -1 4.482096968 ];


using JuMP, Ipopt

ACOPF = Model(solver=IpoptSolver())

nbus = 6
gen_bus_set = 1:3
load_bus_set = 4:6

@variable(ACOPF, Pgen[1:3])  # Active power generation at bus i
@variable(ACOPF, Qgen[1:3])  # Reactive power generation at bus i
@variable(ACOPF, E[i=1:nbus], start = 1.07)     # Real part of the complex voltage at bus i
@variable(ACOPF, F[i=1:nbus], start = 0 )     # Reactive part of the complex voltage at bus i

# -- Slacks
@variable(ACOPF, dG[i=gen_bus_set], start = 1 )
@variable(ACOPF, dR[i=gen_bus_set], start = 1 )

@variable(ACOPF, uG[gen_bus_set], start = 1 )
@variable(ACOPF, lG[gen_bus_set], start = 1 )

@variable(ACOPF, uR[gen_bus_set], start = 1)
@variable(ACOPF, lR[gen_bus_set], start = 1)

@variable(ACOPF, uB[1:nbus], start = 1 )
@variable(ACOPF, lB[1:nbus], start = 1 )


# Power Flow equations :
@NLconstraint(ACOPF, Act_Pwr_Blnc_load_bus[i=load_bus_set],
              sum( -G[i,j]*(E[i]*E[j]+F[i]*F[j]) + B[i,j]*(E[i]*F[j]-F[i]*E[j]) for j=1:nbus ) == Pload[i] )

@NLconstraint(ACOPF, React_Pwr_Blnc_load_bus[i=load_bus_set],
              sum( G[i,j]*(E[i]*F[j]-F[i]*E[j]) + B[i,j]*(E[i]*E[j]+F[i]*F[j]) for j=1:nbus ) ==  Qload[i])


@NLconstraint(ACOPF, Act_Pwr_Blnc_gen_bus[i=gen_bus_set],
              Pgen[i]*dG[i] + sum( -G[i,j]*(E[i]*E[j]+F[i]F[j]) + B[i,j]*(E[i]*F[j]-F[i]*E[j]) for j=1:nbus ) == Pload[i] )

@NLconstraint(ACOPF, React_Pwr_Blnc_gen_bus[i=gen_bus_set],
              Qgen[i]*dR[i] + sum( G[i,j]*(E[i]*F[j]-F[i]*E[j]) + B[i,j]*(E[i]*E[j]+F[i]*F[j]) for j=1:nbus ) == Qload[i] )


# Limits of active and reactive power : 
@NLconstraint(ACOPF, Act_Pwr_Gen_UppLim[i=gen_bus_set], Pgen[i]*dG[i] + uG[i]*uG[i] == Pmax[i] )
@NLconstraint(ACOPF, Act_Pwr_Gen_LwrLim[i=gen_bus_set], Pgen[i]*dG[i] - lG[i]*lG[i] == Pmin[i] )

@NLconstraint(ACOPF, ReAct_Pwr_Gen_UppLim[i=gen_bus_set], Qgen[i]*dR[i] + uR[i]*uR[i] == Qmax[i] )
@NLconstraint(ACOPF, ReAct_Pwr_Gen_LwrLim[i=gen_bus_set], Qgen[i]*dR[i] - lR[i]*lR[i] == Qmin[i] )


# Limits of voltage at each bus : 
@NLconstraint(ACOPF, Voltage_Mag_UppLim[i=1:nbus], E[i]*E[i]+F[i]*F[i] + uB[i]*uB[i] == Vmax[i]*Vmax[i] )
@NLconstraint(ACOPF, Voltage_Mag_LwrLim[i=1:nbus], E[i]*E[i]+F[i]*F[i] - lB[i]*lB[i] == Vmin[i]*Vmin[i] )


# Constraint of reference bus :
@NLconstraint(ACOPF, Slack_Bus_Cons_1, E[1]*E[1] == 1.07*1.07 )
@NLconstraint(ACOPF, Slack_Bus_Cons_2, F[1]*F[1] == 0 )

# Slack variable constraint :
@NLconstraint(ACOPF, Slack_Bus_Cons_3[i=gen_bus_set], dG[i]*dG[i] == 1 )
@NLconstraint(ACOPF, Slack_Bus_Cons_4[i=gen_bus_set], dR[i]*dR[i] == 1 )

# Objective Function :
@NLobjective(ACOPF, Min, sum( c[i]*Pgen[i]*Pgen[i] + b[i]*Pgen[i]*dG[i] + a[i]*dG[i]*dG[i] for i=gen_bus_set ))
# @objective(ACOPF, Min, sum( c[i]*Pgen[i]*Pgen[i] + b[i]*Pgen[i] + a[i] for i=gen_bus_set ))

solve(ACOPF)

println(" ---------------------------- ")
println( "    Real Dispatches = ", getvalue(Pgen*sbase) )
println( "Reactive Dispatches = ", getvalue(Qgen*sbase) )
println( "       Bus Voltages = ", [sqrt(getvalue(E[i]*E[i]+F[i]*F[i])) for i in 1:nbus ] )
println( " Objective Func. Value = ", getobjectivevalue(ACOPF) )

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      236
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      515

Total number of variables............................:       48
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       44
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 